In [37]:
import numpy as np
import matplotlib.pyplot as plt
from image_graph_util import img_conversion_v7
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from emnist import extract_training_samples,extract_test_samples
from emnist import list_datasets
from sklearn.utils import shuffle
import cv2
import glob

In [38]:
img=img_conversion_v7.image_convert()


patch=2


In [39]:
ds, info = tfds.load('CatsVsDogs', split='train', shuffle_files=True, with_info=True)

In [40]:
train_images = []
train_labels = []

for example in ds:
    train_images.append(example['image'].numpy())
    train_labels.append(example['label'].numpy())

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


In [41]:
print(len(train_images))
print(len(train_labels))

23262
23262


In [42]:
images=np.array([cv2.resize(i,(100,100), interpolation= cv2.INTER_CUBIC) for i in train_images])


In [43]:
images = [i/255 for i in images]
labels = np.array(train_labels)

In [44]:
images=np.array(images,dtype="float16")

In [45]:
images.shape

(23262, 100, 100, 3)

In [46]:
train,test,train_lab,test_lab=train_test_split(images,labels,test_size=0.30,random_state=42)


In [47]:
print(train.shape)
print(test.shape)
print(train_lab.shape)
print(test_lab.shape)

(16283, 100, 100, 3)
(6979, 100, 100, 3)
(16283,)
(6979,)


In [48]:
regular_train=train
reg_train_label=train_lab

In [49]:
regular_val,regular_test,reg_val_label,reg_test_label=train_test_split(test,test_lab,test_size=0.50,random_state=42)


In [50]:
print(regular_train.shape)
print(regular_val.shape)
print(regular_test.shape)


(16283, 100, 100, 3)
(3489, 100, 100, 3)
(3490, 100, 100, 3)


In [51]:
from collections import Counter
#print((Counter(y_val)))
print(np.sort(list(Counter(reg_train_label).keys())))
classes=len(np.sort(list(Counter(reg_train_label).keys())))

[0 1]


In [52]:
reg_train_patch=img.to_patcharray(regular_train[:],patch_size=(patch,patch))
reg_val_patch=img.to_patcharray(regular_val[:],patch_size=(patch,patch))
reg_test_patch=img.to_patcharray(regular_test[:],patch_size=(patch,patch))



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [53]:
print(reg_train_patch.shape)
print(reg_val_patch.shape)
print(reg_test_patch.shape)

(16283, 3, 2500, 2, 2)
(3489, 3, 2500, 2, 2)
(3490, 3, 2500, 2, 2)


In [54]:
import networkx as nx


def compute_grid_lap_channel(df_in): #compute_grid_lap_channel
    g = nx.generators.lattice.grid_2d_graph(patch,patch, periodic=False)
    adj_ary=nx.adjacency_matrix(g).toarray()

    edge=np.array(np.where(adj_ary==1))
    ed0=edge[0]
    ed1=edge[1]

    no_img,no_ch, no_patch,patch_x,patch_y=df_in.shape
    temp=[]
    for i in range(no_img):
        i_ary = np.eye(patch_x*patch_y)
        i_ary4d=i_ary[np.newaxis,np.newaxis,:,:]
        weighted_adj=np.zeros((no_ch, no_patch,patch_x*patch_y,patch_x*patch_y))
        df=df_in[i].reshape(no_ch,no_patch,-1)
        pixel1=df[:,:,ed0]
        pixel2=df[:,:,ed1]  
        edu_distance=np.sqrt((pixel1-pixel2)**2)+ 10**(-8)
        weighted_adj[:,:,ed0,ed1]=edu_distance
        sum_r=np.sum(weighted_adj,axis=2)
        temp_adj=sum_r[:,:,:,np.newaxis]
        deg=temp_adj*i_ary4d
        L=deg-weighted_adj
        temp.append(L)
    return temp



In [55]:
from multiprocessing import Pool
import multiprocessing



num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)


Number of CPU cores: 48


In [56]:
train_patch1=np.array_split(reg_train_patch[:], num_cores)
val_patch1=np.array_split(reg_val_patch[:], num_cores)
test_patch1=np.array_split(reg_test_patch[:], num_cores)

In [57]:
print(len(train_patch1))
print(len(val_patch1))
print(len(test_patch1))

48
48
48


In [58]:
num_matrices, num_channels, num_patches, height, width = train_patch1[0].shape

In [59]:
with Pool(processes=num_cores) as pool:
        # Define a list of numbers
    train_p = train_patch1
    
        # Apply the square function to each number using the pool
    g_train=pool.map(compute_grid_lap_channel,train_p)
pool.close()


    
        #results = pool.map(calculate_visibility, numbers)


In [60]:
temp1=[]
for submatrix in g_train:
    temp1.extend(submatrix)


In [61]:
gersh_train= np.array([np.asarray(submatrix) for submatrix in temp1])


In [62]:
print("d")

d


In [63]:
with Pool(processes=num_cores) as pool1:
        # Define a list of numbers
    val_p = val_patch1
    
        # Apply the square function to each number using the pool
    g_val=pool1.map(compute_grid_lap_channel,val_patch1)    
pool1.close()



In [64]:
temp2=[]
for submatrix in g_val:
    temp2.extend(submatrix)


In [65]:
gersh_val= np.array([np.asarray(submatrix) for submatrix in temp2])


In [66]:
print("d")

d


In [67]:
with Pool(processes=num_cores) as pool2:
        # Define a list of numbers
    test_p = test_patch1
    
        # Apply the square function to each number using the pool
    g_test=pool2.map(compute_grid_lap_channel,test_patch1) 
pool2.close()


In [68]:
temp3=[]
for submatrix in g_test:
    temp3.extend(submatrix)


In [69]:
gersh_test= np.array([np.asarray(submatrix) for submatrix in temp3])


gersh_train=np.array(img1.to_gridlaparray(train_patch))
gersh_val=np.array(img1.to_gridlaparray(val_patch))
gersh_test=np.array(img1.to_gridlaparray(test_patch))

In [70]:
print(gersh_train.shape)
print(gersh_val.shape)
print(gersh_test.shape)

(16283, 3, 2500, 4, 4)
(3489, 3, 2500, 4, 4)
(3490, 3, 2500, 4, 4)


In [71]:
lap_train=gersh_train
lap_val=gersh_val
lap_test=gersh_test

In [72]:
print(lap_train.shape)
print(lap_val.shape)
print(lap_test.shape)

(16283, 3, 2500, 4, 4)
(3489, 3, 2500, 4, 4)
(3490, 3, 2500, 4, 4)


In [73]:
rlap_train=np.moveaxis(lap_train,1,-1)
rlap_val=np.moveaxis(lap_val,1,-1)
rlap_test=np.moveaxis(lap_test,1,-1)



In [74]:
rlap_train=rlap_train.reshape(rlap_train.shape[0],rlap_train.shape[1]*rlap_train.shape[2],rlap_train.shape[3],rlap_train.shape[4])
rlap_val=rlap_val.reshape(rlap_val.shape[0],rlap_val.shape[1]*rlap_val.shape[2],rlap_val.shape[3],rlap_val.shape[4])
rlap_test=rlap_test.reshape(rlap_test.shape[0],rlap_test.shape[1]*rlap_test.shape[2],rlap_test.shape[3],rlap_test.shape[4])

In [75]:
print(rlap_train.shape)
print(rlap_train.shape)
print(rlap_train.shape)

(16283, 10000, 4, 3)
(16283, 10000, 4, 3)
(16283, 10000, 4, 3)


# Gershgorien 2D CNN

In [76]:
input_shape = (rlap_train.shape[1],rlap_train.shape[2],rlap_train.shape[3])
num_classes = classes
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Conv2D(filters= 32, kernel_size=(3,3), activation='relu',padding='same',input_shape= input_shape))
model1.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2),strides=None,padding='valid'))
model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),padding='same',activation='relu'))
model1.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 1),strides=None,padding='valid'))

model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),padding='same',activation='relu'))
model1.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 1),strides=None,padding='valid'))

model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(512, activation='relu'))
model1.add(tf.keras.layers.Dropout(0.1))
model1.add(tf.keras.layers.Dense(512, activation='relu'))
model1.add(tf.keras.layers.Dense(num_classes, activation='softmax'))


In [77]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 10000, 4, 32)      896       
                                                                 
 average_pooling2d (AverageP  (None, 5000, 2, 32)      0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 5000, 2, 32)       9248      
                                                                 
 average_pooling2d_1 (Averag  (None, 2500, 2, 32)      0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 2500, 2, 32)       9248      
                                                                 
 average_pooling2d_2 (Averag  (None, 1250, 2, 32)      0

In [78]:
model1.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [79]:
history_log=model1.fit(rlap_train, reg_train_label, epochs=10,verbose=1,validation_data=(rlap_val, reg_val_label))


Epoch 1/10
509/509 [==============================] - 171s 334ms/step - loss: 0.6169 - acc: 0.6560 - val_loss: 0.5753 - val_acc: 0.7091
Epoch 2/10
509/509 [==============================] - 159s 313ms/step - loss: 0.5154 - acc: 0.7461 - val_loss: 0.5524 - val_acc: 0.7211
Epoch 3/10
509/509 [==============================] - 152s 298ms/step - loss: 0.4060 - acc: 0.8121 - val_loss: 0.5899 - val_acc: 0.6976
Epoch 4/10
509/509 [==============================] - 148s 292ms/step - loss: 0.1993 - acc: 0.9190 - val_loss: 0.9178 - val_acc: 0.7120
Epoch 5/10
509/509 [==============================] - 149s 293ms/step - loss: 0.0696 - acc: 0.9749 - val_loss: 1.1910 - val_acc: 0.6993
Epoch 6/10
509/509 [==============================] - 152s 298ms/step - loss: 0.0432 - acc: 0.9851 - val_loss: 1.6812 - val_acc: 0.7034
Epoch 7/10
509/509 [==============================] - 149s 293ms/step - loss: 0.0248 - acc: 0.9931 - val_loss: 1.6934 - val_acc: 0.6988
Epoch 8/10
509/509 [============================

In [80]:
loss, accuracy = model1.evaluate(rlap_test, reg_test_label)
print(f' Model loss on the test set: {loss}')
print(f' Model accuracy on the test set: {100*accuracy}')

110/110 [==============================] - 4s 38ms/step - loss: 2.0021 - acc: 0.7152
 Model loss on the test set: 2.0021164417266846
 Model accuracy on the test set: 71.51862382888794


In [81]:
predictions = model1.predict(rlap_test)
pred_arg = np.argmax(predictions, axis=1)

In [82]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(reg_test_label,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[1256  488]
 [ 506 1240]]


In [83]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [84]:
print(accuracy_score(reg_test_label, pred_arg)*100)
print(f1_score(reg_test_label,pred_arg,average='macro')*100)
print(recall_score(reg_test_label, pred_arg, average="weighted")*100)
print(precision_score(reg_test_label, pred_arg, average="weighted")*100)

71.51862464183381
71.51802601084216
71.51862464183381
71.52108288423656
